# Data Extraction

In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [56]:
import praw

user_agent = "Scraper 1.0 by /u/charltonxqh"
reddit = praw.Reddit(
  client_id="Ijfzy88liEKe7iGMHtKORg",
  client_secret="3kt2_MM5k0AF8PIFXVh32lAr1xEhXg",
  user_agent=user_agent
)